## Porting Micrograd to Julia, Part 1: Building a backpropagation engine

This is a port of Andrej Karpapthy's excellent package `Micrograd` to Julia. See his comprehensive walkthrough for details. A bare-bones exposition is included here.

### Finding the right next step
Fitting models requires a mechanism to determine how to adjust the parameters of the model in order to minimize the difference between the predicted and actual values (loss) or this loss plus a penalty based on model complexity (regularization term).

Autodifferentiation provides a way to find the right way to adjust the parameters. Here we look at *reverse mode* or *backpropagation*.

#### Local derivatives
let's take a look at a simple addition of two values:
```
a = 1
b = 2
c = a + b
```

Derivatives tell us how changes in a or b impact c:
- `dc/da` tells us how `c` will change with `a` change in `a`
- `dc/db` tells us how `c` will change with `a` change in `b`

Here:
```
dc/da = 1 + 0 = 1
dc/da = 0 + 1 = 1
```

what about `f`? how do changes in `d` and `e` change the value of `f`?
```
d = 1
e = 2
f = d*e

df/dd = e = 2 # in this instance because we know the value of e
df/de = d = 1 # plugging in
```

#### propagating derivatives

Models are not just one layer of calucations deep. we need a way to link changes in the output across layers of calculation

Now let's look at `g`:
```
g = c + f
```

How do changes in `a` affect `g`? We can't see this directly, but the chain rule tells us how derivates combine.

```
dg/da = dg/dc * dc/da
        -----   -----
        ^       ^ a's local derivative wrt c
        c's local derivative wrt g
```

This could be arbitrarily deep but each step can be decomposed into the passed derivative and local derivative.

```
loss = f(g(param,X),y) # where f is the loss function and g is the model

d_loss/d_param    =   d_loss/d_1 * d_1/d_2 * d_i/d_i+1 * d_n/d_param
--------------        --------------------------------   -----------
^ goal derivative     ^ passed derivative                ^ local derivative                     
```


So we need the following things to acheive backpropagation and adjust the parameters to monimize the loss:
1. a record of the computation chain from the parameter to the loss
2. the local derivative of each computation
3. a way to combine the local derivative with the passed derivative 

This package implements these.

### Building the components

In [2]:
using Micrograd
# the core type in this implementation is `Value`. It is a numerical value with
# associated attributes that are required to build a computation graph
a = value(1.0)

# it has the following fields. we'll see what they mean
println(a.data) # the numeric value
println(a.grad) # the gradient wrt the target, initially zero
println(a.prev) # children values this one was contructed from, here none since it's a leaf, will need this to build the chain
println(a.op) # operation used in the construction of this value (functions as a label), nothing here
println(a.bw) # backward function, the thing that determines how to combine the passed derivative with the local derivative

# the value prints out in abbreviated from
println(a)

1.0
0.0


()

backward function
1.0 (gr: 0.0, op:  )


In [36]:
# Let's look at a simple computation.
a = value(1.0)
b = value(2.0)
c = a+b
println("The value c, it's gradient and the operation to create it: \n\t c = $c")
println("c is linked to a and b:\n\t c.prev = $(c.prev)")

The value c, it's gradient and the operation to create it: 
	 c = 3.0 (gr: 0.0, op: +)
c is linked to a and b:
	 c.prev = (1.0 (gr: 0.0, op:  ), 2.0 (gr: 0.0, op:  ))


#### Inside an operation
The plus operation looks like this:

```julia
function Base.:+(x::Value{T},y::Value{T}) where T 
    out = value(x.data+y.data,zero(T),(x,y),"+")
    function bwf()
        x.grad += out.grad  # dL/dx = dout/dx * dL/dout = 1.0 (local) * out.grad (passed) = out.grad
        y.grad += out.grad
        return nothing
    end
    out.bw = Backward(bwf)
    return out
end
```

Backpropagation works from parents to children. At the construction of each parent, a specific function is created to combines the local derivative from the operation with the passed derivative from the parent wrt the target to find the gradient of the children wrt the target.

Here's what `*` looks like:
```julia
function Base.:*(x::Value{T},y::Value{T}) where T 
    out = value(x.data*y.data,zero(T),(x,y),"*")
    function bwf()
        x.grad += y.data*out.grad
        y.grad += x.data*out.grad
    end
    out.bw = Backward(bwf)
    return out
end
```

This is exactly what we saw above. The local derivative for one child is the value of the other.

```julia
d = 1
e = 2
f = d*e

df/dd = e = 2 # in this instance because we know the value of e
df/de = d = 1 # plugging in
```

You can work out by hand local derivatives for any function and write the backward function.

```julia
function relu(x::Value{T}) where T
    out_data = x.data > 0.0 ? x.data : 0.0
    out = value(out_data,zero(T),(x,),"relu")
    function bwf()
        x.grad += x.data > 0.0 ? out.grad : 0.0   # dL/dx = dout/dx (1.0 or 0.0) *  dL/dout (out.grad)
    end
    out.bw = Backward(bwf)
    return out
end
```

In [37]:
# we can add a bit more and show the new computation graph
d = tanh(c)
nodes,depth = buildgraph(d)
printgraph(nodes,depth) # see doc string for what this function can and can't do

# note that all gradients are zero


Simple Polytree Viewer: 

1.0 (gr: 0.0, op: tanh)   
|
3.0 (gr: 0.0, op: +)   
|----------------------|
2.0 (gr: 0.0, op:  )   1.0 (gr: 0.0, op:  )   

In [38]:
# we can set the gradient of d and then call it's backward function
d.grad = 1.0
d.bw()
printgraph(nodes,depth)


Simple Polytree Viewer: 

1.0 (gr: 1.0, op: tanh)   
|
3.0 (gr: 0.0099, op: +)   
|----------------------|
2.0 (gr: 0.0, op:  )   1.0 (gr: 0.0, op:  )   

In [39]:
# this propagated the derivative one step back. now call c's backward function
c.bw()
printgraph(nodes,depth)



Simple Polytree Viewer: 

1.0 (gr: 1.0, op: tanh)   
|
3.0 (gr: 0.0099, op: +)   
|-------------------------|
2.0 (gr: 0.0099, op:  )   1.0 (gr: 0.0099, op:  )   

In [40]:
# we can build the computational graph using topological sort such that a node is only added if it's children have already been added
g = buildgraph(d)



(Value[1.0 (gr: 0.0099, op:  ), 2.0 (gr: 0.0099, op:  ), 3.0 (gr: 0.0099, op: +), 1.0 (gr: 1.0, op: tanh)], [3, 3, 2, 1])

In [41]:
# now if we go backward through this list, we will only be passing back gradients that are fully calculated
zerograd(d) # gradients acumulate, need to reset them.
d.grad = 1.0
for n in reverse(nodes)
    n.bw()
end
printgraph(nodes,depth)


Simple Polytree Viewer: 

1.0 (gr: 1.0, op: tanh)   
|
3.0 (gr: 0.0099, op: +)   
|-------------------------|
2.0 (gr: 0.0099, op:  )   1.0 (gr: 0.0099, op:  )   

In [49]:
# now if we want to make d larger we know what to do
a2 = a+a.grad*10 # unrealistically massive step size, but we're in the tail of tanh
b2 = b+b.grad*10
c2 = a2+b2
d2 = tanh(c2)

println("d: $(d.data)")
println("d2: $(d2.data)")

d: 0.9950547536867305
d2: 0.9966646024355773


In [50]:
# or smaller
a2 = a-a.grad*10 # unrealistically massive step size, but we're in the tail of tanh
b2 = b-b.grad*10
c2 = a2+b2
d2 = tanh(c2)

println("d: $(d.data)")
println("d2: $(d2.data)")

d: 0.9950547536867305
d2: 0.9926707543920653


couple this with a gradient descent algorithm and you have a solver!

### A single neuron

In [58]:
## single neuron example, replicating micrograd example
x1 = value(2.0)
w1 = value(-3.0)
x2 = value(0.0)
w2 = value(1.0)
b = value(6.8813735870195432)

x1w1 = x1*w1
x2w2 = x2*w2

x1w1x2w2 = x1w1+x2w2
n = x1w1x2w2+b
o = tanh(n)


println("Pre backpropagation:")
nodes,depth = buildgraph(o)
printgraph(nodes,depth)

println("\n\nPost backpropagation:")

backward(o)
printgraph(nodes,depth)

Pre backpropagation:



Tree:
----- 

0.71 (gr: 0.0, op: tanh)   
|
0.88 (gr: 0.0, op: +)   
|----------------------|
6.9 (gr: 0.0, op:  )   -6.0 (gr: 0.0, op: +)   
                       |---------------------------------------------|
                       0.0 (gr: 0.0, op: *)                          -6.0 (gr: 0.0, op: *)   
                       |----------------------|                      |-----------------------|
                       1.0 (gr: 0.0, op:  )   0.0 (gr: 0.0, op:  )   -3.0 (gr: 0.0, op:  )   2.0 (gr: 0.0, op:  )   

Post backpropagation:

Tree:
----- 

0.71 (gr: 1.0, op: tanh)   
|
0.88 (gr: 0.5, op: +)   
|----------------------|
6.9 (gr: 0.5, op:  )   -6.0 (gr: 0.5, op: +)   
                       |---------------------------------------------|
                       0.0 (gr: 0.5, op: *)                          -6.0 (gr: 0.5, op: *)   
                       |----------------------|                      |-----------------------|
                       1.0 (gr: 0.0, op:  )   0.0 (gr: